In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_diabetes
from datetime import datetime

In [2]:
diabetes=load_diabetes()
data = pd.DataFrame(diabetes['data'], columns=diabetes['feature_names'])
data.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


In [7]:
def cost_function(X, Y, B):
    Y=pd.DataFrame(Y)
    m = len(Y)
    cost = (np.sum((X.dot(B) - Y) ** 2)/(2 * m))[0]
    return cost


def Prediction(X):
    standardize = StandardScaler()
    X_standard = pd.DataFrame(standardize.fit_transform(X))
    X_standard.columns=X.columns.values

    # Adding dummy column for constant coefficient
    const = pd.DataFrame(np.ones(len(X_standard)))
    X_const=pd.concat([const,X_standard],axis=1)
    X_const.columns.values[0]='constant'

    Predict=pd.DataFrame(X_const.dot(np.array(coefficients)))
    return Predict


def R_sq(Actual, Predict):
    Actual_1 = np.array(Actual)
    Predict_1=np.array(Predict)
    SS_res = sum((Actual_1 - Predict_1)**2)
    SS_total = sum((Actual_1 - Actual_1.mean())**2)
    R_sq = 1-(SS_res/SS_total)
    return R_sq

def Linear_model(X_data, y_data):

    time_start = datetime.now()

    #Standardizing the features
    standardize = StandardScaler()
    X_data_standard = pd.DataFrame(standardize.fit_transform(X_data))
    X_data_standard.columns=X_data.columns.values
    X_data =X_data_standard

    # Adding dummy column for constant coefficient
    const = pd.DataFrame(np.ones(len(X_data)))
    X_data=pd.concat([const,X_data],axis=1)
    X_data.columns.values[0]='constant'

    variables = X_data.shape[1]

    global coefficients
    coefficients = np.array([0]*variables)
    y=np.concatenate(np.array(y_data))

    iterations=100
    Learning_rate=0.005 # Learning Rate
    n=len(X_data) # Data points

    delta=[]
    global cost

    cost = [0] * iterations

    for i in range(0,iterations):
        cost[i]=cost_function(X_data, y, coefficients)
        Predicted=np.array(np.dot(X_data,coefficients))
        Error=np.array(Predicted-y)
        delta=(np.array(X_data.T.dot(Error)))
        coefficients=coefficients-delta*(2*Learning_rate/n)

    time_end = datetime.now()
    print('Run Time (Hour:Min:sec)',time_end-time_start,'\n')
    print('Coefficients:',coefficients)
    

In [8]:
X_data=data[['sex','bmi','age','s1','s2','s3','s4','s5','s6']]
y_data=pd.DataFrame(data['bp'])

Linear_model(X_data,y_data)

Run Time (Hour:Min:sec) 0:00:23.353313 

Coefficients: [ 8.14513113e-17  4.76166323e-03  7.66758965e-03  6.71703435e-03
  2.25088685e-03  9.57334109e-05 -4.55042610e-04  4.38484281e-04
  6.21952869e-03  6.66049881e-03]


In [9]:
#Using sklearn model

sklearn_model= LinearRegression()
sklearn_model.fit(X_data, y_data)
print(sklearn_model.intercept_, sklearn_model.coef_)

[5.50022466e-16] [[ 0.17639084  0.24530954  0.17366821  0.23658619 -0.12074717 -0.10736377
  -0.22396394  0.14878206  0.15891835]]


In [10]:
#Model Peroformance (Manual vs Sklearn model)

print('Manual',R_sq(y_data,Prediction(X_data)))
print('Sklearn',R_sq(y_data,sklearn_model.predict(X_data)))

Manual [0.28976762]
Sklearn [0.31480032]
